In [21]:
import textwrap as tw
#wrapper object for the model parameters.
#Simplifies accessing them later at least with my programming style
# one could simply index and array and pass that to everything, but it would harm clarity
class modelparams:
    #object initialization method
        def __init__(self,a,M,L,E):
            self.a=a
            self.M=M
            self.L=L
            self.E=E
    #method for printing values within model object
        def __str__(self):
            selfstring=f"""
            Black Hole Parameters:
            a={self.a} (Rotation Value)
            M={self.M} (Mass Value)
            Light Ray Parameters:
            L={self.L} (Angular Momentum
            E={self.E} (Total Energy)"""
            return tw.dedent(selfstring) #removes strange indents


In [22]:
#black hole params
a=0.5
M=1
#test light params
L=0.5
E=0.5
#init the model object "mdl"
mdl=modelparams(a,M,L,E)

print(mdl)


def Delta(r,model):
    delta=r**2 + model.a**2+
    pass

def P(r,model):
    pass
    
def R(r,model):
    pass



Black Hole Parameters:
a=0.5 (Rotation Value)
M=1 (Mass Value)
Light Ray Parameters:
L=0.5 (Angular Momentum
E=0.5 (Total Energy)
